In [1]:
### Chapter 7: compulsory: 3 and 5 and select one of these two sets: {1} or {2}
from __future__ import division
from __future__ import print_function
import pandas as pd
%pylab inline
import time
import warnings
import os
from collections import Counter
from itertools import chain
import json
import seaborn as sns
import ast
from itertools import izip

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    warnings.filterwarnings("ignore",category=FutureWarning)
import sklearn.preprocessing

import six

from util.VisualizeDataset import VisualizeDataset
from Chapter7.PrepareDatasetForLearning import PrepareDatasetForLearning
from Chapter7.LearningAlgorithms import ClassificationAlgorithms
from Chapter7.LearningAlgorithms import RegressionAlgorithms
from Chapter7.Evaluation import ClassificationEvaluation
from Chapter7.Evaluation import RegressionEvaluation
from Chapter7.FeatureSelection import FeatureSelectionClassification
from Chapter7.FeatureSelection import FeatureSelectionRegression
import copy
from util import util
from sklearn.model_selection import train_test_split
print("Imports done ...✈✈✈✈✈✈✈✈ You are free to go.")




Populating the interactive namespace from numpy and matplotlib
Imports done ...✈✈✈✈✈✈✈✈ You are free to go.


# Chapter 7.3
In our example, we use the same methods, but on a classification task with the following labels, some similar to the crowdsignals dataset.

* Stairs Down
* Stairs Up
* Standing
* Walking
* Sitting
* Running
* On Desk

In [11]:

# Of course we repeat some stuff from Chapter 3, namely to load the dataset

DataViz = VisualizeDataset()

# Read the result from the previous chapter, and make sure the index is of the type datetime.

dataset_path = './intermediate_datafiles/'
export_tree_path = 'Custom_graphs/Chapter7/'

millis = 1000
try:
    dataset = pd.read_csv(dataset_path + str(millis)+'_custom5.csv', index_col=0)
except IOError as e:
    print('File not found, try to run previous crowdsignals scripts first!')
    raise e

if not os.path.exists(export_tree_path):
    os.makedirs(export_tree_path)

dataset.index = dataset.index.to_datetime()

# Let us consider our first task, namely the prediction of the label. We consider this as a non-temporal task.

# We create a single column with the categorical attribute representing our class. Furthermore, we use 70% of our data
# for training and the remaining 30% as an independent test set. We select the sets based on stratified sampling. We remove
# cases where we do not know the label.

prepare = PrepareDatasetForLearning()

train_X, test_X, train_y, test_y = prepare.split_single_dataset_classification(dataset, ['label'], 'like', 0.7, filter=True, temporal=False)
#train_X, test_X, train_y, test_y = prepare.split_single_dataset_classification(dataset, ['label'], 'like', 0.01, filter=True, temporal=False)
print(train_X.shape)
print('Training set length is: ', len(train_X.index))
print('Test set length is: ', len(test_X.index))

# Select subsets of the features that we will consider:

basic_features = ['acc_phone_x','acc_phone_y','acc_phone_z','gyr_phone_x','gyr_phone_y','gyr_phone_z','rot_phone_x','rot_phone_y','rot_phone_z','rot_phone_theta','rot_phone_phi']
pca_features = ['pca_1','pca_2','pca_3','pca_4']
time_features = [name for name in dataset.columns if '_temp_' in name]
freq_features = [name for name in dataset.columns if (('_freq' in name) or ('_pse' in name))]
print('#basic features: ', len(basic_features))
print('#PCA features: ', len(pca_features))
print('#time features: ', len(time_features))
print('#frequency features: ', len(freq_features))
cluster_features = ['cluster']
print('#cluster features: ', len(cluster_features))
features_after_chapter_3 = list(set().union(basic_features, pca_features))
features_after_chapter_4 = list(set().union(basic_features, pca_features, time_features, freq_features))
features_after_chapter_5 = list(set().union(basic_features, pca_features, time_features, freq_features, cluster_features))
print(len(features_after_chapter_3))


/home/cts/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: FutureWarning: to_datetime is deprecated. Use pd.to_datetime(...)


(161, 178)
Training set length is:  161
Test set length is:  70
#basic features:  11
#PCA features:  4
#time features:  30
#frequency features:  99
#cluster features:  1
15


In [ ]:
max_iters = 50
fs = FeatureSelectionClassification()

backward_features, backward_ordered_features, backward_ordered_scores = fs.backward_selection(max_iters, train_X[features_after_chapter_5], train_y)
print('==== Finished backward selection =====')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72


In [15]:
forward_features, forward_ordered_features, forward_ordered_scores = fs.forward_selection(max_iters, train_X[features_after_chapter_5], train_y)

125
